In [2]:
from shared_notebook_utils import *
from statsmodels.sandbox.stats.runs import mcnemar
seaborn.set(style="whitegrid")
%matplotlib inline
datasets = load_datasets(dirnames=['freesound_loops_db_200'], clean=True, exclude_files=['analysis_freesound_extractor_04.json'])

In [19]:
## Edit output of Percival14_bpm according to confidence
def compute_confidence_measure(estimated_bpm,
                       duration_samples,
                       start_effective_duration,
                       end_effective_duration,
                       sample_rate=44100, beat_range=range(1, 128), k=0.5):
    if estimated_bpm == 0:
        # This condition is to skip computing other steps if estimated bpm is 0, we already know that the
        # output will be 0
        return 0

    durations_to_check = [
        duration_samples,
        duration_samples - start_effective_duration,
        end_effective_duration,
        end_effective_duration - start_effective_duration
    ]

    beat_duration = (60.0 * sample_rate)/estimated_bpm
    L = [beat_duration * n for n in beat_range]
    thr_lambda = k * beat_duration
    confidences = list()
    for duration in durations_to_check:
        delta_l = min([abs(l - duration) for l in L])
        if delta_l > thr_lambda:
            confidences.append(0.0)
        else:
            confidences.append(1.0 - float(delta_l) / thr_lambda)
    return max(confidences)


def confidence_helper(estimated_bpm, sound):
    return compute_confidence_measure(
        estimated_bpm,  
        item['analysis']['basic_audio_properties']['length_samples'],
        item['analysis']['basic_audio_properties']['start_effective_duration'], 
        item['analysis']['basic_audio_properties']['end_effective_duration']
    )


# Iterate over all instances in all datasets and for all methods
for dataset in datasets:
    methods_to_compare = ['Percival14_bpm']
    for key, item in dataset.data.items():
        for method in methods_to_compare:
            estimated_bpm = int(round(item['analysis'][method]['bpm_raw']))
            item['analysis'][method]['confidence_ffont'] = confidence_helper(estimated_bpm, item)
            confidences = list()
            WINDOW = 1
            window_range = range(-WINDOW, WINDOW + 1)
            for candidate in [estimated_bpm + x for x in range(-WINDOW, WINDOW + 1)]:
                confidence = confidence_helper(candidate, item)
                confidences.append(confidence)
            
            i = confidences.index(max(confidences))
            final_bpm = estimated_bpm + window_range[i]
            item['analysis'][method + '2'] = dict()
            item['analysis'][method + '2']['bpm_raw'] = final_bpm
            item['analysis'][method + '2']['confidence_ffont'] = confidence_helper(estimated_bpm, item)

In [23]:
for count, dataset in enumerate(datasets):
    print title("\nGeneral tempo estimation results (%s)" % dataset.short_name, c='*')
    methods_to_compare = ['Percival14_bpm', 'Percival14_bpm2']
    table_header = ['Method', 'Accuracy 1e', 'Accuracy 1', 'Accuracy 2']
    table_rows = list()

    for method in methods_to_compare:
        try:
            table_row = [method]
            for accuracy_func in accuracy1e, accuracy1, accuracy2:
                method_results = accuracy_func(dataset.data, method)
                table_row.append(100 * basic_statistics(method_results)['avg'])
            table_rows.append(table_row)
        except IndexError:
            print "Warning: Skipping method %s (analsyis not found in dataset)" % method
            continue
    print ""
    print_table(table_header, table_rows, sort_column=3, highlight_max=True)
    
# Now for confidence above threhsold
thr = 1.0
for count, dataset in enumerate(datasets):
    print title("\nGeneral tempo estimation results (%s) - confidence > %.2f" % (dataset.short_name, thr), c='*')
    methods_to_compare = ['Percival14_bpm', 'Percival14_bpm2']
    table_header = ['Method', 'Accuracy 1e', 'Accuracy 1', 'Accuracy 2']
    table_rows = list()

    for method in methods_to_compare:
        try:
            table_row = [method]
            for accuracy_func in accuracy1e, accuracy1, accuracy2:
                filtered_data = dataset.filter_data([('analysis.%s.%s__>=' % (method, 'confidence_ffont'), thr)]).data
                method_results = accuracy_func(filtered_data, method)
                table_row.append(100 * basic_statistics(method_results)['avg'])
            table_rows.append(table_row)
        except IndexError:
            print "Warning: Skipping method %s (analsyis not found in dataset)" % method
            continue
    print ""
    print_table(table_header, table_rows, sort_column=3, highlight_max=True)    
    



General tempo estimation results (freesound_loops_db_200)
**********************************************************


Method            Accuracy 1e   Accuracy 1   Accuracy 2   
----------------------------------------------------------
Percival14_bpm    53.54         64.14        84.85        
Percival14_bpm2   54.55         64.14        84.85        


General tempo estimation results (freesound_loops_db_200) - confidence > 1.00
******************************************************************************


Method            Accuracy 1e   Accuracy 1   Accuracy 2   
----------------------------------------------------------
Percival14_bpm    62.07         62.07        98.28        
Percival14_bpm2   62.07         62.07        98.28        
